# Extremum Seeking

Two-dimensional Extremum Seeking Algorithm that minimizes or maximizes a single local objective function.

The ES searches over 2 dimensions, with probes that have the same frequency, but are offset by $\pi/2$. The probes may have different amplitudes, and each channel may have a different integrator gain.

In [ ]:
# Imports

import numpy as np
import matplotlib.pyplot as plt

from lib.ExtremumSeekingSimple1D import ExtremumSeekingSimple1D
from lib.ExtremumSeekingSimple2D import ExtremumSeekingSimple2D
from lib.ExtremumSeekingSimpleND import ExtremumSeekingSimpleND

from lib.System_Module import PassThroughSystem

from lib.Objective_Function_Module import ObjectiveFunction

from lib.Simulation_Module import Simulation

from lib.Plotting_Module import plot_es_results



### Example 1: Simulation of ND-ES with simple passthrough system

In this example, a single 2D ESC minimizes the value of an objective function, by estimating the optimal input to an unknown system.

System: $\textbf{y} ( t ) = \textbf{u} ( t )$

Objective function: $\left( \textbf{y} - \textbf{y}^{*} \right)^{T} \left( \textbf{y} - \textbf{y}^{*} \right) = \left( y_{1} - y_{1}^{*} \right)^{2} + \left( y_{2} - y_{2}^{*} \right)^{2}$

Reference: $\textbf{y}^{*} = \left[ 1, -1.5 \right]^{T}$

In [ ]:
# Setup and run simulation of simple passthrough system with multiple ES operating in parallel to minimize/maximize two separate objective functions


# define pass-through system
def pass_through_system_01(u):
    # return np.array([[1.0, 0, 0], [0, 1, 0], [0, 0, 1]])@u
    return np.array([[1.0, 0, 0], [0, 1, 0], [0, 0, 1], [1, 1, 1]])@u

# define objhective function
def objective_function_01(y, ystar):
    return np.sum((y - ystar)**2)

# setup time
dT = 0.01 # simulation timestep
time = np.arange(0,30+dT,dT)

# desired system output (reference signal)
ystar = np.zeros((4,len(time)))
ystar[0,:] = 1
ystar[1,:] = -1
ystar[2,:] = 1.5
ystar[3,:] = 3

# initialize pass-through system object
PST01 = PassThroughSystem(time, dT, 3, 4, pass_through_system_01, ystar)

# initialize objective function object
OBJ01 = ObjectiveFunction(time, dT, objective_function_01, ystar)

# initialize 2D ES
ESC01 = ExtremumSeekingSimpleND(time, dT, 3, [1, 1.1, 1.2], [0.2, 0.2, 0.2], [0.2, 0.2, 0.2], "minimize", name="NDES_01")

# pass list of ES algorithm(s) to system object(s)
PST01.set_es_list([ESC01])

# pass list of system(s) to objective function object(s)
OBJ01.set_system_list([PST01])

# intialize simulation
sim = Simulation(time, dT, [PST01], [ESC01], [OBJ01])

# run simuation
sim.run_simulation()
    
print("Simulation Complete")

# plot results
plot_es_results(time, dT, [sim], [PST01], [OBJ01], [ESC01], OBJ01.ystar)
    

In [ ]:
# Plot results in 2D space

fig1, axs = plt.subplots(1,2,figsize=(24,12))

axs[0].plot(OBJ01.ystar[0,:],OBJ01.ystar[1,:],'.',markersize=10,label='Reference value')
axs[0].plot(PST01.y[0,:],PST01.y[1,:],linewidth=2,label='System output')
axs[0].plot(PST01.y[0,0],PST01.y[1,0],'g.',markersize=10,label='Initial system output')
axs[0].plot(PST01.y[0,-1],PST01.y[1,-1],'r.',markersize=10,label='Final system output')
axs[0].set_title("System output and reference value")
axs[0].set_xlabel(r"$y_{1}$")
axs[0].set_ylabel(r"$y_{2}$")
axs[0].legend()


axs[1].plot(ESC01.thetahat[0,:],ESC01.thetahat[1,:],'--',label='ESC setpoint')
axs[1].plot(ESC01.thetahat[0,0],ESC01.thetahat[1,0],'g.',markersize=10,label='Initial ESC setpoint')
axs[1].plot(ESC01.thetahat[0,-1],ESC01.thetahat[1,-1],'r.',markersize=10,label='Final ESC setpoint')
axs[1].plot(ESC01.theta[0,:],ESC01.theta[1,:],label='ESC control value')
axs[1].plot(ESC01.theta[0,0],ESC01.theta[1,0],'g*',markersize=10,label='Initial ESC control')
axs[1].plot(ESC01.theta[0,-1],ESC01.theta[1,-1],'r*',markersize=10,label='Final ESC control')
axs[1].set_title("ESC setpoint and control values")
axs[1].set_xlabel(r"$\hat{\theta}_{c}$ and $\theta_{c}$")
axs[1].set_ylabel(r"$\hat{\theta}_{s}$ and $\theta_{s}$")
axs[1].legend()

plt.show()